In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
pd.options.display.max_columns = None
pd.options.display.max_rows = None

#got here: https://stackoverflow.com/questions/34398054/ipython-notebook-cell-multiple-outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [8]:
matches = pd.read_csv('matches.csv', low_memory=False)

matches.head()
rows = len(matches.axes[0])
cols = len(matches.axes[1])
print("Number of Rows: ", rows)
print("Number of Columns: ", cols)
matches.columns.values

,id,home,away,date,year,time (utc),attendance,venue,league,part_of_competition,game_status,shootout,home_score,away_score,referee,home_goal_minutes,home_goal_scorers,away_goal_minutes,away_goal_scorers,home_starting_1_num,home_starting_1,home_starting_2_num,home_starting_2,home_starting_3_num,home_starting_3,home_starting_4_num,home_starting_4,home_starting_5_num,home_starting_5,home_starting_6_num,home_starting_6,home_starting_7_num,home_starting_7,home_starting_8_num,home_starting_8,home_starting_9_num,home_starting_9,home_starting_10_num,home_starting_10,home_starting_11_num,home_starting_11,home_bench_1_num,home_bench_1,home_bench_1_minute,home_bench_2_num,home_bench_2,home_bench_2_minute,home_bench_3_num,home_bench_3,home_bench_3_minute,home_bench_4_num,home_bench_4,home_bench_4_minute,home_bench_5_num,home_bench_5,home_bench_5_minute,home_bench_6_num,home_bench_6,home_bench_6_minute,home_bench_7_num,home_bench_7,home_bench_7_minute,home_bench_8_num,home_bench_8,home_bench_8_minute,home_bench_9_num,home_bench_9,home_bench_9_minute,away_starting_1_num,away_starting_1,away_starting_2_num,away_starting_2,away_starting_3_num,away_starting_3,away_starting_4_num,away_starting_4,away_starting_5_num,away_starting_5,away_starting_6_num,away_starting_6,away_starting_7_num,away_starting_7,away_starting_8_num,away_starting_8,away_starting_9_num,away_starting_9,away_starting_10_num,away_starting_10,away_starting_11_num,away_starting_11,away_bench_1_num,away_bench_1,away_bench_1_minute,away_bench_2_num,away_bench_2,away_bench_2_minute,away_bench_3_num,away_bench_3,away_bench_3_minute,away_bench_4_num,away_bench_4,away_bench_4_minute,away_bench_5_num,away_bench_5,away_bench_5_minute,away_bench_6_num,away_bench_6,away_bench_6_minute,away_bench_7_num,away_bench_7,away_bench_7_minute,away_bench_8_num,away_bench_8,away_bench_8_minute,away_bench_9_num,away_bench_9,away_bench_9_minute,home_bench_10_num,home_bench_10,home_bench_10_minute,home_starting_12_num,home_starting_12,home_starting_13_num,home_starting_13,home_starting_14_num,home_starting_14,away_starting_12_num,away_starting_12,away_starting_13_num,away_starting_13,away_starting_14_num,away_starting_14,away_starting_15_num,away_starting_15,home_starting_15_num,home_starting_15,home_starting_16_num,home_starting_16,home_starting_17_num,home_starting_17,home_starting_18_num,home_starting_18,home_starting_19_num,home_starting_19,home_starting_20_num,home_starting_20,home_starting_21_num,home_starting_21,home_starting_22_num,home_starting_22,home_bench_11_num,home_bench_11,home_bench_11_minute,home_bench_12_num,home_bench_12,home_bench_12_minute,home_bench_13_num,home_bench_13,home_bench_13_minute,home_bench_14_num,home_bench_14,home_bench_14_minute,away_starting_16_num,away_starting_16,away_starting_17_num,away_starting_17,away_starting_18_num,away_starting_18,away_starting_19_num,away_starting_19,away_starting_20_num,away_starting_20,away_starting_21_num,away_starting_21,away_bench_10_num,away_bench_10,away_bench_10_minute,away_bench_11_num,away_bench_11,away_bench_11_minute,away_bench_12_num,away_bench_12,away_bench_12_minute,away_starting_22_num,away_starting_22,away_bench_13_num,away_bench_13,away_bench_13_minute,home_possessionPct,away_possessionPct,home_shotsSummary,away_shotsSummary,home_foulsCommitted,away_foulsCommitted,home_yellowCards,away_yellowCards,home_redCards,away_redCards,home_offsides,away_offsides,home_wonCorners,away_wonCorners,home_saves,away_saves,away_bench_14_num,away_bench_14,away_bench_14_minute,home_formation,away_formation
0,NaN,New England,San Jose,7/31/1996,1996,NaN,"12,871",Foxboro Stadium,1996 MLS,Regular Season,FT,NaN,2,0,Ruben Rodhas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

Number of Rows:  7289
Number of Columns:  209


array(['id', 'home', 'away', 'date', 'year', 'time (utc)', 'attendance',
       'venue', 'league', 'part_of_competition', 'game_status',
       'shootout', 'home_score', 'away_score', 'referee',
       'home_goal_minutes', 'home_goal_scorers', 'away_goal_minutes',
       'away_goal_scorers', 'home_starting_1_num', 'home_starting_1',
       'home_starting_2_num', 'home_starting_2', 'home_starting_3_num',
       'home_starting_3', 'home_starting_4_num', 'home_starting_4',
       'home_starting_5_num', 'home_starting_5', 'home_starting_6_num',
       'home_starting_6', 'home_starting_7_num', 'home_starting_7',
       'home_starting_8_num', 'home_starting_8', 'home_starting_9_num',
       'home_starting_9', 'home_starting_10_num', 'home_starting_10',
       'home_starting_11_num', 'home_starting_11', 'home_bench_1_num',
       'home_bench_1', 'home_bench_1_minute', 'home_bench_2_num',
       'home_bench_2', 'home_bench_2_minute', 'home_bench_3_num',
       'home_bench_3', 'home_bench_3_min